# Lab 8 - Bayesian Inference

In the 8th lab of the course, we will study Bayesian Inference in practice.

We will explore the use of Bayesian inference thought a Decision Making example.

# Assignment 1: Bayesian Decision Making

## N Meteorologies Problem

In this assignment, we explore the challenge of dealing with predictions from N different meteorologies, each forecasting whether it will rain the next day. Specifically, each meteorology predicts the likelihood of rain for the following day.

To be more precise, we can interpret each meteorology as a model that predicts whether it will rain or not. Mathematically, this is expressed as $P(y | M_i)$ for each $i \in M$, where $M$ is the set of possible meteorologies. Here, the random variable $y$ indicates whether it will rain or not (where $y = 1$ means rain and $y = 0$ means no rain). The output of each model, $P(y = 1 | M_i)$, represents the probability of rain for the next day.

For the purposes of this exercise, we assume that we have $N = 3$ different meteorologies for simplicity.  
The set $M$ is defined as $M = \{M1, M2, M3\}$.


# Step 1.1 Marginal Probability

Let's assume that, for the first day, we have no clue about which meteorology is the best. In the Bayesian interpretation, this means our prior belief is the same for every meteorology. In other words, our belief is $P(M_i) = 1/N$ for every $i \in M$.

Furthermore, the predicted probability of rain for the different meteorological stations, $P(y=1|M)$, is given by the following list of numbers: $[0.1, 0.5, 0.7]$.

One way to produce our final estimation of rain is to use the Bayesian Marginal prediction:

$$p_{\text{marginal}}(y) =  \sum_{i \in M} p(y \mid M_i) \cdot P(M_i)$$

The marginal prediction is the average of the predictions of each meteorology weighted by our belief.  
In other words, it is weighted by how much we trust each meteorology.

After obtaining the marginal prediction $p_{\text{marginal}}(y)$, we can select our final prediction (or action) $a$ that maximizes the marginal prediction:

$$a = \arg \max_{y} \; p_{\text{marginal}}(y)$$


**Complete the following steps**:  
A) Define a vector with the initial prior belief over the models p(M).  
B) Create a function to calculate the marginal prediction $p_{\text{marginal}}(y)$ for spesific y.   
C) Make your final action (or prediction) that maximisise the Bayesian marginal prediction.

In [2]:
def get_marginal_prediction(y, prior, p_y_m):
    n_y = p_y_m.shape[0] 
    
    ## marginal_prediction
    marginal_prediction = sum(p_y_m[y,:] * prior)
        
    return marginal_prediction

In [3]:
import numpy as np
propability_rain_model = np.array([0.1, 0.5, 0.7]) # p( y=1 | M )
propability_model = np.array([1-propability_rain_model, propability_rain_model]) # p(y|M)

# A) define prior
n_model = propability_rain_model.shape[0] # fill your code
prior = np.ones(n_model)/n_model # fill your code


# B) calculate marginal prediction
p_y_1_marginal = get_marginal_prediction(y= 1, p_y_m=propability_model, prior=prior)
p_y_marginal = np.array([1-p_y_1_marginal, p_y_1_marginal])

# C) calculate the prediction that maximise the marginal predictions
final_actions = np.argmax(p_y_marginal)

In [4]:
propability_model

array([[0.9, 0.5, 0.3],
       [0.1, 0.5, 0.7]])

In [5]:
print("Our final prediction is :", final_actions)

Our final prediction is : 0


# Step 1.2 Decision According to Utility

In some applications, our decisions (or actions) significantly affect people.

Consider the case of wrongly predicting the weather. Wrongly predicting bad weather affects people less. In contrast, wrongly predicting good weather may have a significant impact on people.

One way to adjust our actions according to the effect on the user is to define an additional utility function $U(a, y)$ that outputs a scalar indicating how much our final actions are affected by the outcome $y$.

We can then select the action $a$ that maximizes the expected utility $u(a) = E_{y \sim p_m(y)}[U(a,y)]$ according to our model $p_m$ to estimate the outcome $y$.

The expected utility is defined as follows:
$$ u(a) = E_{y \sim p_m(y)}[U(a,y)] = \sum_y U[a,y] p_m(y) $$

And the final action $a^{\star}$ is the one that maximizes the expected utility:
$$ a^{\star} = \arg \max_{y} \;  u(a) = \arg \max_{y} \;  \sum_y U[a,y] p_m(y)  $$

In our example, we have the following utility function $U[a,y]$:

$$
U[a,y] = \begin{bmatrix}
           1  & -10 \\
          -1 & 1
        \end{bmatrix}
$$

So, if our prediction (or action) is correct, i.e., $a = y$ (diagonal of the matrix), then we get a utility of 1.  
If our action is 0 (no rain) and it's actually raining ($y=1$), then we incur a big penalty of -10.  
If our action is 1 (rain) and it's actually not raining ($y=0$), then we incur a small penalty.


**Complete the following steps**:  
A) Fill the function bellow, that calculates the expected utility of an action $a$ aconding to a utility function U, and a model $p_m$  
B) Produce the final action $a^{\star}$ that maximise the expected utility acording to the marginal model, and the utility function defined above.  
C) Comment of the result.

In [6]:
def get_expected_utility(action , model , U):
    """
    Calculates the expected utility of an action a, according to a model, for specific utility function U
    """
    n_outcomes = model.shape[0]
    utility = 0
    for x in range(n_outcomes):
        utility +=  U[action, x] * model[x]
        
    return utility

In [7]:
U = np.array([[1, -10],
              [-1, 1]])

In [8]:
# find the best 
u_a_1 = get_expected_utility(action=1, model=p_y_marginal, U=U)
u_a_0 = get_expected_utility(action=0, model=p_y_marginal, U=U)

final_actions_utility = np.argmax(np.array([u_a_0, u_a_1]))

In [9]:
p_y_marginal.shape

(2,)

In [10]:
print("Our final prediction is :", final_actions_utility)

Our final prediction is : 1


# Step 1.3 Updating Belief

After collecting our data $D$, in our example, if it actually rains ($y_{\text{true}}$), it makes sense to update our belief $p(M)$ about the best model.

Using the Bayesian interpretation, we can update the belief $p(M)$ by calculating the posterior distribution:

$$ p(M_i|y_{\text{true}}) = \frac{p(y_{\text{true}}|M_i) \cdot p(M_i)}{p(y_{\text{true}})} $$

$$ = \frac{p(y_{\text{true}}|M_i) \cdot p(M_i)}{\sum_{j \in M} p(y_{\text{true}}|M_j) \cdot p(M_j)} $$

Then, until we observe some new data, we can use the posterior as our new prior $p(M)$.


**Complete the following steps**:  
A) Create a function that calculate the posterior propability distribution.  
B) Calculate the posterior is the case that the true outcome is y = 1 (rain).  
C) Compare the vulues of the posterior with the values of the prior, comment on the results.  
D) Set the prior to be equal to the posterior, in order to update your belief for the next predictions.  

In [11]:
# A)
def get_posterior(prior, P, outcome):
    """
    Calculate the posterior given a prior belief, a set of predictions, an outcome
    - prior: belief vector so that prior[i] is the probabiltiy of mdoel i being correct
    - P: p(y|m) P[y][m] is the probability the m-th model assignsm to the y-th outcome
    - outcome: actual outcome
    """

    n_models = len(prior)
    ## fill in
    posterior = np.zeros(n_models)
    posterior = P[outcome, :] * prior / sum(P[outcome, :] * prior)
    return posterior

In [12]:
y_true = 1
prior

array([0.33333333, 0.33333333, 0.33333333])

In [13]:
# B) calculate the posterior, based on the true outcome, and the old model
posterior = get_posterior(prior=prior, P = propability_model, outcome = y_true)

array([[0.9, 0.5, 0.3],
       [0.1, 0.5, 0.7]])

In [13]:
# C) compare prior and posterior
print("Prior distribution:", prior)
print("Posterior distribution:", posterior)

# try to add some comment? what you observe?

Prior distribution: [0.33333333 0.33333333 0.33333333]
Posterior distribution: [0.07692308 0.38461538 0.53846154]


In [14]:
# D) update prior
prior = posterior

# Step 1.3 Sequential desition making

To wrap up everything above, consider the case that we sequentialy have to estimation our actions $a$ based on our belief about the model p(m). 

For 3 consequtive days we get sequential prediction from the different meteorologies.
After each day we also observe the true outcome, so we update our prior to make the action of the next day.

The predictions and the true outcome is given in the followin code block.



**Complete the following steps**:  
Iterate over the different days and:
A) Calculate the marginal prediction  
B) Select the action that magimise the marginal prediction  
C) Select the action the maximise the expected utility based on the marginal model  
D) Update the prior using the posterior of the true outcome  
E) Comment on the final results

In [18]:
T = 3 # number of time steps
n_models = 3 # number of models

# build predictions for each station of rain probability
predictions = np.array( 
                       [[0.1, 0.4, 0.7], # day 1
                        [0.1, 0.1, 0.8], # day 2
                        [0.3, 0.6, 0.9]] # day 3
                      )


true_y = [0, 0, 1];
n_outcomes = 2 # 0 = no rain, 1 = rain

In [16]:
## we use this matrix to fill in the predictions of stations
for t in range(T):
    # unify predictions
    propability_rain_model = predictions[t]
    propability_model = np.array([1-propability_rain_model, propability_rain_model]) # p(y|M)
    
    # get marginal model
    p_y_1_marginal = get_marginal_prediction(y= 1, p_y_m=propability_model, prior=prior)
    p_y_marginal = np.array( [ 1 - p_y_1_marginal, p_y_1_marginal ] )
    
    # Action that maximise the marginal model
    action_marginal = np.argmax(p_y_marginal)
    
    # Action expected utility based on marginal model
    u_a_1 = get_expected_utility(action=1, model=p_y_marginal, U=U)
    u_a_0 = get_expected_utility(action=0, model=p_y_marginal, U=U)

    final_actions_utility = np.argmax(np.array([u_a_0, u_a_1]))
    
    # update
    old_prior = prior
    posterior = get_posterior(prior=prior, P = propability_model, outcome = true_y[t])
    prior = posterior
    
    
    print(f"-------iteration day {t}")
    print("Prior  p(m) = ", old_prior)
    print("Predictions p(y=1|m) = ",predictions[t] )
    print("True outcome = ",true_y[t] )
    print("Marginal prediction, p_marginal(y=1) = ", p_y_1_marginal)
    print("Action that maximise the marginal model:", action_marginal)
    print("Action that maximise the expected utility U according to the marginal model:", final_actions_utility)
    print("Posterior:", posterior)
    print("\n")

-------iteration day 0
Prior  p(m) =  [0.07692308 0.38461538 0.53846154]
Predictions p(y=1|m) =  [0.1 0.4 0.7]
True outcome =  0
Marginal prediction, p_marginal(y=1) =  0.5384615384615384
Action that maximise the marginal model: 1
Action that maximise the expected utility U according to the marginal model: 1
Posterior: [0.15 0.5  0.35]


-------iteration day 1
Prior  p(m) =  [0.15 0.5  0.35]
Predictions p(y=1|m) =  [0.1 0.1 0.8]
True outcome =  0
Marginal prediction, p_marginal(y=1) =  0.345
Action that maximise the marginal model: 0
Action that maximise the expected utility U according to the marginal model: 1
Posterior: [0.20610687 0.6870229  0.10687023]


-------iteration day 2
Prior  p(m) =  [0.20610687 0.6870229  0.10687023]
Predictions p(y=1|m) =  [0.3 0.6 0.9]
True outcome =  1
Marginal prediction, p_marginal(y=1) =  0.5702290076335879
Action that maximise the marginal model: 1
Action that maximise the expected utility U according to the marginal model: 1
Posterior: [0.10843373 

**E) What is the best model according to the posterior distribution after process?**

# Step 1.4 Decision Based on Maximum a posteriori (MAP)

Another way to make decisions is to select the model that performs the best according to our posterior distribution.

More specifically, in each step, we can choose the model that maximizes the posterior:
      $$m^{\star} = \arg \max_{m} p(m|Data) $$

And then obtain the best action according to that model $p(m|Data)$ instead of the marginal model.
1. Obtain the action with the maximum probability according to the best model $p(y|m^{\star})$.
2. The second option is to select the action that maximizes the expected utility based on the best model $p(y|m^{\star})$.


**Complete the following steps**:   
A) Select the model $m^{\star}$ with the maximum posterior (MAP estimator)   
B) Calculate the action that with the maximum propability according to the model $p(y|m^{\star})$  
C) Calculate the action that with the maximum expected utility, according to the model $p(y|m^{\star} )$  

In [44]:
predictions = [0.3, 0.3, 0.6]

In [45]:
map_estimator = np.argmax(posterior)

In [46]:
p_y_1 =  predictions[map_estimator] # p(y=1|m*)
p_y_map = np.array( [ 1 - p_y_1, p_y_1 ] )

In [47]:
action_map = np.argmax(p_y_map)
action_map

0

In [48]:
u_a_1 = get_expected_utility(action=1, model=p_y_map, U=U)
u_a_0 = get_expected_utility(action=0, model=p_y_map, U=U)

final_actions_utility = np.argmax(np.array([u_a_0, u_a_1]))
final_actions_utility

1